In [299]:
import os
import pandas as pd

In [309]:
folder_path = '/Users/akshat/Documents/Thapar/Projects/Cricket Analysis/MAIN/Dataset'

match_summary = pd.DataFrame()
batting_summary = pd.DataFrame()
bowling_summary = pd.DataFrame()

for year_folder in os.listdir(folder_path):
    year_folder_path = os.path.join(folder_path, year_folder)

    if os.path.isdir(year_folder_path):
        match = [file for file in os.listdir(year_folder_path) if file.startswith('df1')]
        batting = [file for file in os.listdir(year_folder_path) if file.startswith('df2')]
        bowling = [file for file in os.listdir(year_folder_path) if file.startswith('df3')]

        for file in match:
            file_path = os.path.join(year_folder_path, file)

            df = pd.read_csv(file_path)
            df['Year'] = int(year_folder)

            match_summary = pd.concat((match_summary, df), ignore_index=True)
            
        for file in batting:
            file_path = os.path.join(year_folder_path, file)

            df = pd.read_csv(file_path)
            df['Year'] = int(year_folder)

            batting_summary = pd.concat((batting_summary, df), ignore_index=True)
            
        for file in bowling:
            file_path = os.path.join(year_folder_path, file)

            df = pd.read_csv(file_path)
            df['Year'] = int(year_folder)

            bowling_summary = pd.concat((bowling_summary, df), ignore_index=True)

match_summary = match_summary.iloc[:, 1:]
match_summary = match_summary[['Year'] + [col for col in match_summary.columns if col != 'Year']]
match_summary = match_summary.reset_index(drop=True)

batting_summary = batting_summary.iloc[:, 1:]
batting_summary = batting_summary[['Year'] + [col for col in batting_summary.columns if col != 'Year']]
batting_summary = batting_summary.reset_index(drop=True)

bowling_summary = bowling_summary.iloc[:, 1:]
bowling_summary = bowling_summary[['Year'] + [col for col in bowling_summary.columns if col != 'Year']]
bowling_summary = bowling_summary.reset_index(drop=True)

In [310]:
match_summary = match_summary.dropna(subset=['Winners'])
match_summary['Date'] = match_summary['Date'].fillna(method='ffill')
match_summary.reset_index(drop=True, inplace=True)

match_summary['Winner'] = match_summary['Winners'].str.split('won').str[0]

In [311]:
batting_summary['Date'] = batting_summary['Date'].str.strip().str.rstrip(',')
batting_summary['Stadium'] = batting_summary['Stadium'].str.strip().str.rstrip(',')

bowling_summary['Date'] = bowling_summary['Date'].str.strip().str.rstrip(',')
bowling_summary['Stadium'] = bowling_summary['Stadium'].str.strip().str.rstrip(',')

In [312]:
batting_summary['Date'] = batting_summary['Date'].astype(str) + str(' ') + batting_summary['Year'].astype(str)
batting_summary['Date'] = pd.to_datetime(batting_summary['Date'], format='%b %d %Y')

bowling_summary['Date'] = bowling_summary['Date'].astype(str) + str(' ') + bowling_summary['Year'].astype(str)
bowling_summary['Date'] = pd.to_datetime(bowling_summary['Date'], format='%b %d %Y')

match_summary['Date'] = match_summary['Date'].str.split(',').str[0]
match_summary['Date'] = match_summary['Date'].astype(str) + str(' ') + match_summary['Year'].astype(str)
match_summary['Date'] = pd.to_datetime(match_summary['Date'], format='%b %d %Y')

In [313]:
batting_summary['Match Details'] = batting_summary['Match Details'].str.replace("- Live Cricket Score, Commentary","").str.strip()
batting_summary[['Team 1', 'Team 2']] =batting_summary['Match Details'].str.split(',').str[0].str.split(' vs ', expand=True)

bowling_summary['Match Details'] = bowling_summary['Match Details'].str.replace("- Live Cricket Score, Commentary","").str.strip()
bowling_summary[['Team 1', 'Team 2']] =bowling_summary['Match Details'].str.split(',').str[0].str.split(' vs ', expand=True)

match_summary[['Team 1', 'Team 2']] =match_summary['Teams'].str.split(',').str[0].str.split(' vs ', expand=True)

In [314]:
batting_summary = batting_summary[['Year', 'Date', 'Time', 'Stadium', 'City', 'Match Details', 'Team 1', 'Team 2', 'Team Playing', 'Total Runs', 'Total Wickets', 'Batsman Names', 'Out/Not Out', 'Balls Played', 'Runs Scored', 'Fours', 'Sixes', 'Strike Rate']]
batting_summary = batting_summary.rename(columns = {'Team Playing' : 'Team Batting', 'Year' : 'IPL Edition'})

bowling_summary = bowling_summary[['Year', 'Date', 'Time', 'Stadium', 'City', 'Match Details', 'Team 1', 'Team 2', 'Team Bowling', 'Total Runs Conceeded', 'Total Wickets Taken', 'Bowler Name', 'Overs', 'Maidens', 'Runs', 'Wickets', 'No Balls', 'Wides', 'Economies']]
bowling_summary = bowling_summary.rename(columns = {'Year' : 'IPL Edition'})

match_summary = match_summary[['Year', 'Date', 'Time', 'Teams','Team 1', 'Team 2','Winner', 'Winners']]
match_summary = match_summary.rename(columns = {'Teams' : 'Match Details', 'Year' : 'IPL Edition', 'Winners': 'Winning Details'} )

In [315]:
match_summary = match_summary.sort_values(by=['IPL Edition', 'Date'], ascending=[False, True])
batting_summary = batting_summary.sort_values(by=['IPL Edition', 'Date'], ascending=[False, True])
bowling_summary = bowling_summary.sort_values(by=['IPL Edition', 'Date'], ascending=[False, True])

In [316]:
output_folder_path = '/Users/akshat/Documents/Thapar/Projects/Cricket Analysis/MAIN/Dataset'
match_summary.to_csv(os.path.join(output_folder_path, 'match_summary.csv'), index=False)
batting_summary.to_csv(os.path.join(output_folder_path, 'batting_summary.csv'), index=False)
bowling_summary.to_csv(os.path.join(output_folder_path, 'bowling_summary.csv'), index=False)

In [317]:
import os

folder_path = '/Users/akshat/Documents/Thapar/Projects/Cricket Analysis/MAIN/Dataset'

for year_folder in os.listdir(folder_path):
    year_folder_path = os.path.join(folder_path, year_folder)

    if os.path.isdir(year_folder_path):
        for file in os.listdir(year_folder_path):
            if(file.startswith('df1')):
                old_path = os.path.join(year_folder_path, file)
                year = file.split('_')[1].split('.')[0]
                df = pd.read_csv(old_path)
                df = df.drop('Links', axis = 1)
                df.to_csv(os.path.join(year_folder_path, f"df1_{year}.csv"), index = False)
                year = file.split('_')[1].split('.')[0]
                new_name = f"match_summary_{year}.csv"
                new_path = os.path.join(year_folder_path, new_name)
                os.rename(old_path, new_path)
                
            if(file.startswith('df2')):
                old_path = os.path.join(year_folder_path, file)
                year = file.split('_')[1].split('.')[0]
                new_name = f"batting_summary_{year}.csv"
                new_path = os.path.join(year_folder_path, new_name)
                os.rename(old_path, new_path)
                
            if(file.startswith('df3')):
                old_path = os.path.join(year_folder_path, file)
                year = file.split('_')[1].split('.')[0]
                new_name = f"bowling_summary_{year}.csv"
                new_path = os.path.join(year_folder_path, new_name)
                os.rename(old_path, new_path)
                